# Ostrofsky's rootN times rootN PIR scheme based on Learning with Errors and Regev Encryption :

Using parameters from the paper: **One server for the price of Two: Simple and Fast Single-Server Private Information Retreival**

Here the number of LWE samples(m) = m

In [1]:
import numpy as np


# parameters
n = 2**10
m = 10 # sqrt(N), number of samples

error_values = {
    "loc": 0,
    "sigma": 6.4,
}
q = 2**23
p = 991 # plaintext modulus
scaling_factor = np.floor(q/p)


print(f"scaling_factor = floor(q/p) = {np.floor(q/p)}")

x = np.zeros(m)
x[1]=1
print(x * scaling_factor)
print(f"x = {x}")

db = np.random.randint(0, p, (m,m))
print(f"db = {db}")

scaling_factor = floor(q/p) = 8464.0
[   0. 8464.    0.    0.    0.    0.    0.    0.    0.    0.]
x = [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
db = [[854 966  95 838 248 876 408 332 394 543]
 [ 13 726 161 541 139 707 343  30  28 763]
 [704 481 671 543 213 616  51 166 706 149]
 [ 86 628 372 301 979 919 919 349 770 959]
 [154 763 428 319  14 246 552  97 606 771]
 [170 607 508 158  45 754 836 646 234 658]
 [435 149 682 134 898 530 430 255  69 250]
 [521 521 790 650 649 621 385 480 792 882]
 [847 864 753 660 874  61 217 714 186  11]
 [110 508 909 511 530  64 662 567 550 944]]


In [2]:
#using matrices (when you have m samples)


def sampleLWE(m,n,q,error_values, s):
    A= np.random.randint(0, q, (m,n))
    
    e= np.random.normal(error_values["loc"], error_values["sigma"], (m) )

    print(f"A = {A}")
    print(f"s = {s}")
    # print(f"e = {e}")


    # Computing the dot product and adding the error

    # Outputting the vectors a, s and the value b

    b = (np.dot(A,s)  + e ) % q
    print(f"b = As + e mod q = {b}")
    b_prime = (np.dot(A,s) % q + e)


    return A, b_prime




In [3]:
# print(f"A = ", A)
# print(type(A), A.shape)
# print(f"s = ", s)
# print(type(s), s.shape)
# print(f"e = ", e)
# print(type(e), e.shape)

In [4]:
secret= np.random.randint(0, q, (n))

A, b = sampleLWE(m,n,q,error_values, secret)

A = [[3532177 1091081 3446668 ... 3330815 2565741 2878772]
 [6865052 3693163 5907124 ... 5588628 5314853 2323205]
 [1952124 6634517  514713 ... 1046282 7262642 6288454]
 ...
 [6546718 1352470 7265633 ... 8345914 4572747 5647770]
 [7773654 6401648 1834570 ... 6575348 6428763 2487190]
 [5437943 6355535 4090662 ... 6116866 3653500 3432956]]
s = [3336880 1384367 2928838 ...  608071 5547207 4627919]
b = As + e mod q = [2486068. 7986488. 2905592. 3323872. 5057212. 6646520. 5584152. 2208124.
 3493320. 3178768.]


In [5]:
# print(f"as mod q = {np.dot(A, s)% q}")

# print(f"(b = As mod q + e ) mod q = ", b)

# print(f" b_prime = As + e mod q    = {b_prime}")

In [6]:
# Encryption

def regevEncrypt(b, x, p, q):
    scaling_factor = np.floor(q/p)
    c =( b + scaling_factor*x ) 
    c_mod_q = c % q

    

    # print(f"c= As + e + (floor(q/p) * x) \n \\ \
    # = As + e + {scaling_factor}*{x} = {c} = {c_mod_q}")

    return c_mod_q

In [7]:


c = regevEncrypt(b, x, p, q)
print(c)

[2486068.55181588 7994950.78853992 2905591.63712138 3323871.62870821
 5057211.01771312 6646520.6555271  5584151.40213928 2208125.21756655
 3493320.77348278 3178770.67012963]


Send Client --- (mu_enc = A, c) ----> Server

In [8]:
# server computations:

# D dot A

# do you reduce the entire thing modulo q or at the end only

DA = np.dot(db,A) % q
Dc = np.dot(db,c) % q

print(DA)
print(Dc)

[[4569027 1290456 5628061 ...  244682  561836 5030340]
 [1068981 3205415 4976589 ... 7889867 2460143 1622580]
 [4729709 4669933 4126824 ... 5304611 6513511 4864606]
 ...
 [  13280 2632944 3045854 ... 4196291 8147312 7238969]
 [4604963 7586510 7499812 ... 7780761 1345059 3214006]
 [4097948 5141071  236154 ... 5539716 7322709 7688346]]
[1113776.93645859 7580325.70825958 1977674.79397964 1487992.41289139
 2905305.87447739 2918226.1872139  3281273.93385696 1000387.79800415
  589624.70074081   86819.62728119]


In [9]:
#Decryption

# print(np.dot(a, s), np.dot(a,s) % q)

def rounding(x, round_off_to):
    """
    Rounds off a number x to the nearest multiple of round_off_to.

    :param x: The number to be rounded off.
    :param round_off_to: The nearest multiple to which x needs to be rounded off.
    :return: The rounded off value.
    """
    return np.round(x / round_off_to) * round_off_to

# def regevDecrpt(A, s, q, p, c):
#     scaling_factor = np.floor(q/p)
#     # a_s_mod_q = np.dot(A,s)%q 
#     # print(f"a_s_modq = as mod q = ", a_s_mod_q)

#     d_hat = (c - np.dot(A, s)) % q 

#     print(f"d_hat = c - as mod q= {c} - {np.dot(A,s)} mod q = ", d_hat)
#     rounded_d = rounding(d_hat, scaling_factor)



#     print(f"rounded d = {rounded_d}")
    
#     d = rounded_d / scaling_factor
#     print(f"d = rounded_d/ floor(q/p)= {rounded_d}/ {scaling_factor} = {d}" )
#     return d






In [10]:
print(DA)
print(Dc)

[[4569027 1290456 5628061 ...  244682  561836 5030340]
 [1068981 3205415 4976589 ... 7889867 2460143 1622580]
 [4729709 4669933 4126824 ... 5304611 6513511 4864606]
 ...
 [  13280 2632944 3045854 ... 4196291 8147312 7238969]
 [4604963 7586510 7499812 ... 7780761 1345059 3214006]
 [4097948 5141071  236154 ... 5539716 7322709 7688346]]
[1113776.93645859 7580325.70825958 1977674.79397964 1487992.41289139
 2905305.87447739 2918226.1872139  3281273.93385696 1000387.79800415
  589624.70074081   86819.62728119]


In [11]:
d_hat = ( Dc[0] - np.dot(DA[0], secret) ) % q
print(f"d_hat = {d_hat}")

d_hat_prime = ( Dc[0] - ( np.dot(DA[0], secret) % q  ) ) % q
print(f"d_hat_prime = {d_hat}")

rounded_d = rounding(d_hat_prime, scaling_factor)



print(f"rounded d = {rounded_d}")

d = rounded_d / scaling_factor
print(f"d = rounded_d/ floor(q/p)= {rounded_d}/ {scaling_factor} = {d}" )

print(db)

d_hat = 8176584.0
d_hat_prime = 8176584.0
rounded d = 8176224.0
d = rounded_d/ floor(q/p)= 8176224.0/ 8464.0 = 966.0
[[854 966  95 838 248 876 408 332 394 543]
 [ 13 726 161 541 139 707 343  30  28 763]
 [704 481 671 543 213 616  51 166 706 149]
 [ 86 628 372 301 979 919 919 349 770 959]
 [154 763 428 319  14 246 552  97 606 771]
 [170 607 508 158  45 754 836 646 234 658]
 [435 149 682 134 898 530 430 255  69 250]
 [521 521 790 650 649 621 385 480 792 882]
 [847 864 753 660 874  61 217 714 186  11]
 [110 508 909 511 530  64 662 567 550 944]]


In [12]:
# print(f"x = {x}")
# d = regevDecrpt(A, secret, q, p, c = Dc[0])